In [ ]:
cd ..

In [ ]:
import echofilter.raw

In [ ]:
import os

import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
turbulence_color = "c"
bottom_color = "#00ee00"

In [ ]:
ROOT_DATA_DIR = "/media/scott/scratch/Datasets/dsforce/surveyExports"

sample = "mobile/Survey17/Survey17_GR4_T1W_E"

# example with a gap
sample = "mobile/Survey16/Survey16_GR2_S3W_E"  # 115 apart = 3.47m
sample = "mobile/Survey16/Survey16_GR2_N1A_F"  # 045 apart = 1.36m

fname_raw = os.path.join(ROOT_DATA_DIR, sample + "_Sv_raw.csv")
fname_masked = os.path.join(ROOT_DATA_DIR, sample + "_Sv.csv")

ts_raw, depths_raw, signals_raw = echofilter.raw.loader.transect_loader(fname_raw)
ts_mskd, depths_mskd, signals_mskd = echofilter.raw.loader.transect_loader(fname_masked)

t_bot, d_bot = echofilter.raw.loader.evl_loader(
    os.path.join(ROOT_DATA_DIR, sample + "_bottom.evl")
)
t_top, d_top = echofilter.raw.loader.evl_loader(
    os.path.join(ROOT_DATA_DIR, sample + "_turbulence.evl")
)

In [ ]:
ts_new, d_top_new, d_bot_new = echofilter.raw.manipulate.make_lines_from_masked_csv(
    fname_masked
)

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_mskd, depths_mskd, signals_mskd.T)
plt.plot(ts_new, d_top_new, turbulence_color)
plt.plot(ts_new, d_bot_new, bottom_color)
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Zoomed in on top
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_mskd[-450:], depths_mskd[:500], signals_mskd[-450:, :500].T)
plt.plot(ts_new[-450:], d_top_new[-450:], turbulence_color)
# plt.plot(ts_new, d_bot_new, bottom_color)
plt.gca().invert_yaxis()
plt.show()

### Check the resulting masks agree

In [ ]:
ddepths = np.broadcast_to(depths_raw, signals_raw.shape)
mask_top = np.single(ddepths < np.expand_dims(d_top_new, -1))
mask_bot = np.single(ddepths > np.expand_dims(d_bot_new, -1))
mask_new = ~np.any([mask_top, mask_bot], axis=0)

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_mskd, depths_mskd[:2000], ~np.isnan(signals_mskd)[:, :2000].T)
plt.plot(t_top, d_top, turbulence_color)
plt.plot(t_bot, d_bot, bottom_color)
plt.gca().invert_yaxis()
plt.title("Original mask and lines")
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(ts_mskd, depths_mskd[:2000], mask_new[:, :2000].T)
plt.plot(ts_new, d_top_new, turbulence_color)
plt.plot(ts_new, d_bot_new, bottom_color)
plt.gca().invert_yaxis()
plt.title("New mask and lines")
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(
    ts_mskd, depths_mskd[:2000], (mask_new != ~np.isnan(signals_mskd))[:, :2000].T
)
# plt.plot(ts_new, d_top_new, turbulence_color)
# plt.plot(ts_new, d_bot_new, bottom_color)
plt.gca().invert_yaxis()
plt.title("Mask difference")
plt.show()